<h1 align="center">evaluation</h1>

In [ ]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
import glob
import operator
import pandas as pd
import math
%matplotlib inline

In [ ]:
def load_coords(pred_path, gt_path):
    with open(pred_path + '\\test_image' + '-x.json', "r") as fp:
        pred_x = json.load(fp)

    with open(pred_path + '\\test_image' + '-y.json', "r") as fp:
        pred_y = json.load(fp)

    with open(gt_path + '\\test_image' + '-x.json', "r") as fp:
        gt_x = json.load(fp)

    with open(gt_path + '\\test_image' + '-y.json', "r") as fp:
        gt_y = json.load(fp)
        
    return pred_x, pred_y, gt_x, gt_y

In [ ]:
def find_delta(module, pred_path, gt_path):
    
    pred_x, pred_y, gt_x, gt_y = load_coords(pred_path, gt_path)
    
    # define pixel/mm ratio for the image
    if module == 1:
        mm_px_r = 18/239
        
    if module == 2:
        mm_px_r = 18/158
        
    if module == 3:
        mm_px_r = 18/217
        
    if module == 4:
        mm_px_r = 18/257
    
    # define mm threshold for a successful prediction
    min_delta_mm = 1.5


    delta_px = 0
    failed_gt_idx = []
    failed_pred_idx = []
    
    
# find fails
    
    for i in range(len(gt_x)):

        smallest_delta = 1e6

        for j in range(len(pred_x)):
            # find the absolute difference in x
            x_delta = abs(gt_x[i] - pred_x[j])
            # find the absolute difference in y
            y_delta = abs(gt_y[i] - pred_y[j])
            # find euclidian distance of difference
            euclidian_delta = math.sqrt(x_delta**2 + y_delta**2)
            # keep the smallest distance
            if euclidian_delta < smallest_delta:
                prediction_index = j
                smallest_delta = euclidian_delta
                
        # flag if the smallest distance is greater than the threshold in mm
        if smallest_delta * mm_px_r > min_delta_mm:
            failed_gt_idx.append(i)
    
            if smallest_delta * mm_px_r < 10:
#                 print('predictions for ground truth point', i, 'failed by prediction', prediction_index)
                failed_pred_idx.append(prediction_index)
            else:
                failed_pred_idx.append(-1)
    
    # find px distance
    
    for j in range(len(pred_x)):
        smallest_delta = 1e6
        
        for i in range(len(gt_x)):
            # find the absolute difference in x
            x_delta = abs(gt_x[i] - pred_x[j])
            # find the absolute difference in y
            y_delta = abs(gt_y[i] - pred_y[j])
            # find euclidian distance of difference
            euclidian_delta = math.sqrt(x_delta**2 + y_delta**2)
            # keep the smallest distance
            if euclidian_delta < smallest_delta:
                smallest_delta = euclidian_delta
                
                # add the smallest distance to the total delta metric
        delta_px += smallest_delta
    
    delta_mm = delta_px * mm_px_r
    
    # difference in number of predicted spots to real spots
    delta_spots = abs(len(pred_x) - len(gt_x))
    
    print('sum of errors in px=', delta_px,
         '\nmissed/extra spots =', delta_spots,
         '\nfails =', len(failed_gt_idx))
    
    return failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots

In [ ]:
iterables = [["method_1", "method_2"], ["module_1", "module_2", "module_3", "module_4"]]

df = pd.DataFrame(index = pd.MultiIndex.from_product(iterables, names=["method", "module"]),
                  columns = ['failed_predictions', 'pixel_delta', 'mm_delta', 'spot_count_delta'])

cwd = r"C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\evaluation"

for method_dir in glob.glob(cwd+'/*module*/*method*'):
    gt_path = '\\'.join(method_dir.split('\\')[:-1]) + '\\ground-truth'
    module_dir = method_dir.split('\\')[-2]
    method = method_dir.split('\\')[-1][0:8]
    
    print('\n\n' + module_dir, ',', method_dir.split('\\')[-1])
    
    if module_dir == 'module_1':
        module = 1
    elif module_dir == 'module_2':
        module = 2
    elif module_dir == 'module_3':
        module = 3
    elif module_dir == 'module_4':
        module = 4
    
    try:
        failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots = find_delta(module, method_dir, gt_path)
        fails = len(failed_gt_idx)
        results = [fails, delta_px, delta_mm, delta_spots]
        
        df.loc[(method, module_dir), :] = results
        
    except FileNotFoundError:
        print('No predictions for', module_dir, ',', method_dir.split('\\')[-1])
        
df       

#### Visually validate the failures

In [ ]:
method = 2
module = 4

pred_path = f'C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_{module}\\method_{method}_predicted\\'
gt_path = f'C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_{module}\\ground-truth'
image_path = f'C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_{module}\\test_image.jpg'

failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots = find_delta(module, pred_path, gt_path)

pred_x, pred_y, gt_x, gt_y = load_coords(pred_path, gt_path)

In [ ]:
pred_x.append(None)
pred_y.append(None)

In [ ]:
gt_xs = [gt_x[i] for i in failed_gt_idx]
pred_xs = [pred_x[i] for i in failed_pred_idx]
gt_ys = [gt_y[i] for i in failed_gt_idx]
pred_ys = [pred_y[i] for i in failed_pred_idx]

In [ ]:
from PIL import Image
im = Image.open(image_path).convert('L')
arr = np.asarray(im)

plt.scatter(gt_xs,gt_ys, c='b', marker="x", alpha=0.7, linewidth=0.5)
plt.scatter(pred_xs,pred_ys, c='r', marker="x", alpha=0.7, linewidth=0.5)
plt.imshow(arr, cmap='gray', vmin=0, vmax=255)
plt.savefig(pred_path + 'failed_predictions' + '.png', dpi=500)

#### Human trials

In [ ]:
def load_coords(pred_path, gt_path):
    with open(pred_path + '-x.json', "r") as fp:
        pred_x = json.load(fp)

    with open(pred_path + '-y.json', "r") as fp:
        pred_y = json.load(fp)

    with open(gt_path + 'test_image' + '-x.json', "r") as fp:
        gt_x = json.load(fp)

    with open(gt_path + 'test_image' + '-y.json', "r") as fp:
        gt_y = json.load(fp)
        
    return pred_x, pred_y, gt_x, gt_y

In [ ]:
df = pd.DataFrame(columns = ['module', 'failed_predictions', 'pixel_delta', 'mm_delta', 'spot_count_delta']).set_index('module')

for module in range(1,5):
    
    print(f'\nmodule {module}')
    pred_path = f'C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\human_trials\\module_{module}'
    gt_path = f'C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_{module}\\ground-truth\\'

    failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots = find_delta(module, pred_path, gt_path)
    fails = len(failed_gt_idx)
    results = [fails, delta_px, delta_mm, delta_spots]
        
    df.loc[f'module {module}', :] = results
    
df

In [ ]:
module = 2
print(f'\nmodule {module}')
pred_path = f'C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\human_trials\\felix_module_{module}'
gt_path = f'C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_{module}\\ground-truth\\'

failed_gt_idx, failed_pred_idx, delta_px, delta_mm, delta_spots = find_delta(module, pred_path, gt_path)
fails = len(failed_gt_idx)
results = [fails, delta_px, delta_mm, delta_spots]

df.loc[f'module {module}', :] = results